In [1]:
'''
  Install packages required to connect to MySQL
  instance deployed in GCP. The connection is via
  Cloud SQL service offered by GCP
  NOTE: A session restart is required after installation
  of the cloud-sql-python-connector module
'''
# Install PyMySQL python module
!pip install cloud-sql-python-connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.29.0 which is incompatible.


In [1]:
'''
  Install the python MySQL module used for
  managing connections to MySQL DB from
  a python runtime
'''
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.0 MB/s eta 0:00:00


In [4]:
'''
  Configure the environment with the GCP
  project to which the MySQL instance is
  assigned.
'''
from tools import load_db_cfg
dbcfg = load_db_cfg('cfg.json')
#project_id="youtubechannelanalysis-423709"
!gcloud config set project {dbcfg['project_id']}

Updated property [core/project].


In [5]:
'''
  Aunthenticate to Google Colab as a valid
  GCP user that has access to the above
  project and its resources.
  NOTE: The project needs to be pre-configured
  for the user access
'''
from google.colab import auth
auth.authenticate_user()


In [7]:
import pandas as pd
from ytdbconnect import  YtDbConnector
import sqlalchemy

'''
  Data Access Object (DAO) that could be used to store and load
  channel, playlist, comments & vieos
'''
class YtDao:
  sql = {
      'add_channel':'''INSERT INTO channel (id,name,type,views,description,status,playlist_id) VALUES (:id,:name,:type,:views,:description,:status,:playlist_id);''',
      'get_channel':'''SELECT id 'Channel Id',name 'Channel Name',type 'Channel Type',views 'View Count',description 'Description',status 'Status',playlist_id 'Playlist Id' FROM channel WHERE id=%(id)s;''',
      'get_supported_channels':'''SELECT id,name,description FROM channel ORDER BY name;''',
      'add_playlist':'''INSERT INTO playlist (id,channel_id,name) VALUES (:id,:channel_id,:name);''',
      'get_playlists_in_channel': '''SELECT id 'Playlist Id',name 'Playlist Name',channel_id 'Channel Id' FROM playlist WHERE channel_id=%(channel_id)s;''',
      'add_comment':'''INSERT INTO comment (id,video_id,author,published_date,comment_text) VALUES (:id,:video_id,:author,STR_TO_DATE(:published_date,"%Y-%m-%dT%H:%i:%sZ"),:comment_text);''',
      'get_comments_in_channel':'''SELECT id 'Comment Id',video_id 'Video Id',comment_text 'Comment Text',author Author,published_date 'Published Date' FROM comment WHERE video_id IN (SELECT id FROM video WHERE channel_id=%(channel_id)s);''',
      'add_video':'''INSERT INTO video (id,playlist_id,channel_id,name,description,published_date,view_count,like_count,dislike_count,favorite_count,comment_count,duration,caption_status) VALUES (:id,:playlist_id,:channel_id,:name,:description,STR_TO_DATE(:published_date,"%Y-%m-%dT%H:%i:%sZ"),:view_count,:like_count,:dislike_count,:favorite_count,:comment_count,:duration,:caption_status);''',
      'get_videos_in_channel':'''SELECT id 'Video Id',playlist_id 'Playlist Id',channel_id 'Channel Id',name 'Video Name',description 'Video Description', published_date 'Published Date',view_count 'View Count',like_count 'Like Count',dislike_count 'Dislike Count',favorite_count 'Favourite Count',comment_count 'Comment Count',duration 'Duration (secs)',caption_status 'Caption Status' FROM video WHERE video.playlist_id IN (SELECT playlist_id FROM channel WHERE channel_id=%(channel_id)s);''',
      'add_thumbnail':'''INSERT INTO thumbnails_in_video (video_id,type,url) VALUES (":video_id",":type",":url");''',
      'get_thumbnails_in_video': '''SELECT video_id,type,url,video_id FROM thumbnails_in_video WHERE video_id IN (SELECT id from video WHERE channel_id=%(channel_id)s));''',

  }

  __supported_stats__ = {
      'S1':{'description':'What are the names of all the videos and their corresponding channels?',
            'sql':'''SELECT v.name 'Video Name',c.name 'Channel Name' FROM video v JOIN channel c ON (v.channel_id=c.id);'''},
      'S2':{'description':'Which channels have the most number of videos, and how many videos do they have?',
            'sql':'''SELECT c.id 'Channel Id',c.name 'Channel Name',count(v.id) 'Video Count' FROM channel c JOIN video v ON (c.id=v.channel_id) GROUP BY c.id,c.name ORDER BY 'Video Count' DESC'''},
      'S3':{'description':'What are the top 10 most viewed videos and their respective channels?',
            'sql':'''SELECT v.id 'Video Id',v.name 'Video Name',v.view_count 'View Count',c.name 'Channel Name' FROM video v JOIN channel c ON (v.channel_id=c.id) ORDER BY view_count DESC LIMIT 10'''},
      'S4':{'description':'How many comments were made on each video, and what are their corresponding video names?',
            'sql':'''SELECT id 'Video Id', name 'Video Name', comment_count 'Comment Count' FROM video ORDER BY comment_count DESC'''},
      'S5':{'description':'Which videos have the highest number of likes, and what are their corresponding channel names?',
            'sql':'''SELECT v.id 'Video Id',v.name 'Video Name', v.like_count 'Likes Count',c.name 'Channel Name' FROM video v JOIN channel c ON (v.channel_id=c.id) ORDER BY v.like_count DESC LIMIT 50;'''},
      'S6':{'description':'What is the total number of likes and dislikes for each video, and what are their corresponding video names?',
            'sql':'''SELECT id 'Video Id', name 'Video Name', like_count 'Likes Count', dislike_count 'Dislikes Count' FROM video ORDER BY like_count DESC'''},
      'S7':{'description':'What is the total number of views for each channel, and what are their corresponding channel names?',
            'sql':'''SELECT id 'Channel Id',name 'Channel Name',views 'Channel Views' FROM channel ORDER BY views DESC'''},
      'S8':{'description':'What are the names of all the channels that have published videos in the year 2022',
            'sql':'''SELECT c.id 'Channel Id',c.name 'Channel Name' FROM channel c WHERE c.id IN (SELECT video.channel_id FROM video WHERE YEAR(published_date)=2022);'''},
      'S9':{'description':'What is the average duration of all videos in each channel, and what are their corresponding channel names?',
            'sql':'''SELECT v.channel_id 'Channel Id',c.name 'Channel Name',AVG(v.duration) 'Avg. Duration (secs)' FROM video v JOIN channel c ON (v.channel_id=c.id) GROUP BY v.channel_id, c.name ORDER BY AVG(v.duration) DESC;'''},
      'S10':{'description':'Which videos have the highest number of comments, and what are their corresponding channel names?',
             'sql':'''SELECT v.id 'Video Id',v.name 'Video Name',c.name 'Channel Name', v.comment_count 'Comment Count' FROM video v JOIN channel c ON(v.channel_id=c.id) ORDER BY comment_count DESC'''}
  }

  def __init__(self,dbcfg):
    self.__ytd_connector__ = YtDbConnector(dbcfg)
    self.__connection__ = self.__ytd_connector__.connect()

  def is_connected(self):
    if self.__connection__ is None:
      return False
    else:
      return True

  '''
    Get all the statistics that are supported by this DAO
  '''
  def get_supported_stats(self):
    ret = dict()
    df = pd.DataFrame(self.__supported_stats__)
    return df.T['description']
    #for key in self.__supported_stats__.keys():
     # ret[key]=self.__supported_stats__[key]['description']
    #return ret


  def get_stat(self, stat_key):
    df=pd.read_sql_query(self.__supported_stats__[stat_key]['sql'], self.__connection__)
    return df


  '''
    Insert channel information and retun the inserted information
    as a dataframe
  '''
  def __add_channel__(self,channel):
    stmt = sqlalchemy.text(self.sql['add_channel'],)
    self.__connection__.execute(stmt,{'id': channel['id'], 'name': channel['name'], 'type': channel['type'], 'views': channel['views'], 'description': channel['description'], 'status': channel['status'],'playlist_id':channel['playlistId']})

  def add_channel(self,channel):
      self.__add_channel__(channel)
      self.__connection__.commit()
      df=self.get_channel(channel['id'])
      return df


  '''
    Retrieve channel information for the specified channel and
    return it as a dataframe
  '''
  def get_channel(self, channel_id):
      df=pd.read_sql_query(self.sql['get_channel'], self.__connection__,params={'id':channel_id})
      return df

  '''
    Retrieve the list of channels that are already in the db
  '''
  def get_supported_channels(self):
      df=pd.read_sql_query(self.sql['get_supported_channels'], self.__connection__)
      return df

  '''
    Insert playlists for the specified channel and retun inserted info in
    a dataframe
  '''
  def __add_playlists__(self,channel_id,playlists):
    stmt = sqlalchemy.text(self.sql['add_playlist'],)
    for key in playlists.keys():
      playlist=playlists[key]
      self.__connection__.execute(stmt,{'id':key, 'name': playlist['name'], 'channel_id': channel_id})

  def add_playlists(self,channel_id, playlists):
    self.__add_playlists__(channel_id,playlists)
    self.__connection__.commit()
    df = self.get_playlists_by_channel(channel_id)
    return df

  '''
    Retrieve all playlists for the specified channel and
    return them in dataframe
  '''
  def get_playlists_by_channel(self, channel_id):
    return pd.read_sql_query(self.sql['get_playlists_in_channel'], self.__connection__, params={'channel_id':channel_id})

  '''
    Insert comment threads for the specified channel and retun inserted info in
    a dataframe
  '''
  def __add_comments__(self,channel_id, comments):
    stmt = sqlalchemy.text(self.sql['add_comment'],)
    for key in comments.keys():
      comment=comments[key]
      self.__connection__.execute(stmt,{'id': comment['id'],'video_id': comment['video_id'],'author': comment['author'],'published_date': comment['published_date'],'comment_text':comment['text']})

  def add_comments(self,channel_id, comments):
    self.__add_comments__(channel_id, comments)
    self.__connection__.commit()
    df=self.get_comments_by_channel(channel_id)
    return df

  '''
    Retrieve all comment threds for the specified channel and
    return them in dataframe
  '''
  def get_comments_by_channel(self, channel_id):
    return pd.read_sql_query(self.sql['get_comments_in_channel'], self.__connection__, params={'channel_id':channel_id})

  '''
    Insert video info for the specified channel and retun inserted info in
    a dataframe
  '''
  def __add_videos__(self,channel_id, videos):
    stmt = sqlalchemy.text(self.sql['add_video'],)
    for key in videos.keys():
      video=videos[key]
      self.__connection__.execute(stmt,{'id':video['id'],'playlist_id':video['playlist_id'],'channel_id':video['channel_id'],'name':video['name'],'description':video['description'],'published_date':video['published_date'],'view_count':video['view_count'],'like_count':video['like_count'],'dislike_count':video['dislike_count'],'favorite_count':video['favorite_count'],'comment_count':video['comment_count'],'duration':video['duration'],'caption_status':video['caption_status']})

  def add_videos(self,channel_id, videos):
    self.__add_videos__(channel_id, videos)
    self.__connection__.commit()
    df=self.get_videos_by_channel(channel_id)
    return df

  '''
    Retrieve all video for the specified channel and
    return them in dataframe
  '''
  def get_videos_by_channel(self, channel_id):
    return pd.read_sql_query(self.sql['get_videos_in_channel'], self.__connection__, params={'channel_id': channel_id},index_col="id")

  def __add_thumbnails__(self,videos):
    stmt = sqlalchemy.text(self.sql['add_thumbnail'],)
    for video in videos.values():
      for key in video['thumbnails'].keys():
        url= video['thumbnails'][key]
        self.__connection__.execute(stmt,{'video_id': video['id'],'type': key,'url': url})

  def get_thumbnails_in_channel(self,channel_id):
    return pd.read_sql_query(self.sql['get_thumbnails_in_channel'], self.__connection__, params={'channel_id': channel_id},index_col="video_id")

  def close(self):
    self.__connection__.close()


  '''
    Add all the channel information in oneshot
  '''
  def add_channel_info(self, channel, playlists, comments, videos):
    self.__add_channel__(channel)
    self.__add_playlists__(channel['id'],playlists)
    self.__add_videos__(channel['id'], videos)
    self.__add_comments__(channel['id'], comments)
    #self.__add_thumbnails__(videos)
    self.__connection__.commit()







In [14]:
import traceback
from tools import load_db_cfg
'''
  Test program for retrieving the stats from DB
'''
try:
  dbcfg= load_db_cfg('cfg.json')
  dao = YtDao(dbcfg)
  stat_key='S10'
  df_ss= dao.get_supported_stats()
  print(f'''\n{dao.get_stat(stat_key).to_markdown()}''')
  print(f'''\n Supported Stats: {dao.get_supported_stats().to_markdown()}''')
  print(f'''\n Supported Channels:\n {dao.get_supported_channels().to_markdown()}''')
except:
  print(f'''Failed to get stats for key {stat_key}''')
  print(traceback.print_exc())

print('\nConnection closed .. !')
dao.close()

Successfully connected to 'youtube' database!! 

|    | Video Id    | Video Name                                                                                             | Channel Name    |   Comment Count |
|---:|:------------|:-------------------------------------------------------------------------------------------------------|:----------------|----------------:|
|  0 | da1vvigy5tQ | Reversing Type 2 diabetes starts with ignoring the guidelines | Sarah Hallberg | TEDxPurdueU           | TEDx Talks      |           20488 |
|  1 | tPEU-ze_C54 | 'You're lying': George Conway clashes with Republican commentator over Trump guilty verdict            | CNN             |           18305 |
|  2 | cSasPIDo-J8 | Is Trump headed to jail after guilty verdict? Hear what legal expert thinks                            | CNN             |            8241 |
|  3 | GTqOaVv8m1E | தரமான சம்பவம் செய்யும் DRDO | ஜெர்மனியை எதிரியாக மாற்றிய இந்தியாவின் துரோகி | Tamil Pokkisham |Vicky                    

In [ ]:
'''
  YtDao Tests verifying individual table commits
'''
import traceback
from tools import load_db_cfg
test_channels={
    'Tamil Pokkisham': 'UCS84kz7Fs8bzRs6xcPY9lQQ',
    'CNN': 'UCupvZG-5ko_eiXAupbDfxWw',
    "TEDx Talks":'UCsT0YIqwnpJCM-mx7-gSA4Q',
    "WION":"UC_gUM8rL-Lrg6O3adPW9K1g",
    "The Indian Mystics":"UCcnBJHMugWpuy6Y7-SlI1ew",
    "Future Technology":"UCoIPNbr4UXhhZXDBZaUNHwA",
    "SonyMusicSouthVEVO":"UCTNtRdBAiZtHP9w7JinzfUg",
    "NBA":"UCWJ2lWNubArHWmf3FIHbfcQ",
    "moneycontrol":"UChftTVI0QJmyXkajQYt2tiQ",
    "GUVI":"UCduIoIMfD8tT3KoU0-zBRgQ",
    "Neural networks":"UCYO_jab_esuFRV4b17AJtAw"
}

# Mock Data
channel_id=''
channel=playlists=videos=comments={}
# End Mock Data
try:
  dao = YtDao(load_db_cfg('cfg.json'))
  try:
    print(f'''\n{dao.add_channel(channel).to_markdown()}''')
    try:
      print(f'''\n{dao.add_playlists(channel_id,playlists).to_markdown()}''')
      try:
          print(f'''\n{dao.add_videos(channel_id,videos).to_markdown()}''')
          try:
            print(f'''\n{dao.add_comments(channel_id,comments).to_markdown()}''')
          except:
            print('add_comments() failed.')
            print(traceback.print_exc())
      except:
          print('add_videos() failed.')
          print(traceback.print_exc())
    except:
      print('add_playlists() failed.')
      print(traceback.print_exc())
  except:
    print('add_channel() failed')
    print(traceback.print_exc())

except:
  print('Connector init failed.')
  print(traceback.print_exc())



print('Connection closed .. !')
dao.close()


In [13]:
'''
  YtDao Tests verifying oneshot commit
'''
import traceback
test_channels={
    'Tamil Pokkisham': 'UCS84kz7Fs8bzRs6xcPY9lQQ',
    'CNN': 'UCupvZG-5ko_eiXAupbDfxWw',
    "TEDx Talks":'UCsT0YIqwnpJCM-mx7-gSA4Q',
    "WION":"UC_gUM8rL-Lrg6O3adPW9K1g",
    "The Indian Mystics":"UCcnBJHMugWpuy6Y7-SlI1ew",
    "Future Technology":"UCoIPNbr4UXhhZXDBZaUNHwA",
    "SonyMusicSouthVEVO":"UCTNtRdBAiZtHP9w7JinzfUg",
    "NBA":"UCWJ2lWNubArHWmf3FIHbfcQ",
    "moneycontrol":"UChftTVI0QJmyXkajQYt2tiQ",
    "GUVI":"UCduIoIMfD8tT3KoU0-zBRgQ",
    "Neural networks":"UCYO_jab_esuFRV4b17AJtAw"
}

# Mock data
#channel_id=test_channels['TEDx Talks']
channel={'id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'TEDx Talks', 'type': 'youtube#channel', 'description': "TEDx is an international community that organizes TED-style events anywhere and everywhere -- celebrating locally-driven ideas and elevating them to a global stage. TEDx events are produced independently of TED conferences, each event curates speakers on their own, but based on TED's format and rules.\n\nFor more information on using TED for commercial purposes (e.g. employee learning, in a film or online course), please submit a Media Request here: https://media-requests.ted.com.", 'playlistId': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'views': '7872107568', 'hidden_subscribers': False, 'subscriber_count': '40800000', 'status': 'public', 'video_count': '216394'}
playlists={'PLsRNoUx8w3rNyeKk_xNnclpAqrWQztioB': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Plus! TEDx talks en Français', 'id': 'PLsRNoUx8w3rNyeKk_xNnclpAqrWQztioB'}, 'PLsRNoUx8w3rOoNARU75dai0eJbHrm6MzD': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Più! TEDx talks in Italiano', 'id': 'PLsRNoUx8w3rOoNARU75dai0eJbHrm6MzD'}, 'PLsRNoUx8w3rOwHx4kVJL5ksS9vTxj5hXn': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Mais! TEDx talks em Português', 'id': 'PLsRNoUx8w3rOwHx4kVJL5ksS9vTxj5hXn'}, 'PLsRNoUx8w3rPI8m9mqZKrHbGWVn2agSB9': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Looking for unusual (dream) jobs?', 'id': 'PLsRNoUx8w3rPI8m9mqZKrHbGWVn2agSB9'}, 'PLsRNoUx8w3rP7Uu5QuOdYJFfbaKhdHons': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Studying the mystery and wonder of outer space', 'id': 'PLsRNoUx8w3rP7Uu5QuOdYJFfbaKhdHons'}, 'PLsRNoUx8w3rOERmePNyC2hAONG9tL2ZYx': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': "Editor's Picks", 'id': 'PLsRNoUx8w3rOERmePNyC2hAONG9tL2ZYx'}, 'PLsRNoUx8w3rMNeOOWhAlkryhcv5Z33uL3': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'TED Countdown', 'id': 'PLsRNoUx8w3rMNeOOWhAlkryhcv5Z33uL3'}, 'PLsRNoUx8w3rOfyAmUHG3M5MZRUH9kURWv': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'TEDx talks in Romanian', 'id': 'PLsRNoUx8w3rOfyAmUHG3M5MZRUH9kURWv'}, 'PLsRNoUx8w3rNG2nd-8FFqvz3j382QBdCB': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'TEDx talks in Thai', 'id': 'PLsRNoUx8w3rNG2nd-8FFqvz3j382QBdCB'}, 'PLsRNoUx8w3rMMsM9C6ExbJk-cAE3FWdVM': {'channelId': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'TEDx talks in Hungarian', 'id': 'PLsRNoUx8w3rMMsM9C6ExbJk-cAE3FWdVM'}}
comments={'Ugw5rY37mvayy11_LlF4AaABAg': {'id': 'Ugw5rY37mvayy11_LlF4AaABAg', 'video_id': 'CPCWO5177KI', 'text': 'Very motivational, just at the right time!', 'author': '@nunojohn8520', 'published_date': '2024-05-31T17:29:29Z'}, 'UgxIou_zpqjWR13_Pi94AaABAg': {'id': 'UgxIou_zpqjWR13_Pi94AaABAg', 'video_id': 'CPCWO5177KI', 'text': '<a href="https://www.youtube.com/watch?v=CPCWO5177KI&amp;t=646">10:46</a> Such an interesting point', 'author': '@georgebarnesby360', 'published_date': '2024-05-31T17:28:11Z'}, 'UgwfR0CNb5pcCmUkI8J4AaABAg': {'id': 'UgwfR0CNb5pcCmUkI8J4AaABAg', 'video_id': 'V6yixyiJcos', 'text': '&quot;Math isn&#39;t hard, it&#39;s a language&quot;<br>Ok....so... it is a hard language???', 'author': '@OhUiginn', 'published_date': '2024-05-31T17:27:51Z'}, 'UgyIWtsj3L4oMnpqbQh4AaABAg': {'id': 'UgyIWtsj3L4oMnpqbQh4AaABAg', 'video_id': '3icoSeGqQtY', 'text': 'What does any of those things has to do with Math?', 'author': '@OhUiginn', 'published_date': '2024-05-31T17:27:04Z'}, 'UgwgbeIBPrc8d3dcI2V4AaABAg': {'id': 'UgwgbeIBPrc8d3dcI2V4AaABAg', 'video_id': '-l_oc8Ocrig', 'text': 'Weak presentation - I&#39;d rather having a scientist or doctor on the subject.', 'author': '@danibitt59', 'published_date': '2024-05-31T17:26:35Z'}, 'Ugz7EM3k6MLOb8M-SrN4AaABAg': {'id': 'Ugz7EM3k6MLOb8M-SrN4AaABAg', 'video_id': 'GsFzT6IA_hU', 'text': 'Hi is one of  the biggest imbeciles in the entire world 😂😂😂😂', 'author': '@lopabarpaganski', 'published_date': '2024-05-31T17:26:13Z'}, 'UgwV1vUx_t739Rg7fi94AaABAg': {'id': 'UgwV1vUx_t739Rg7fi94AaABAg', 'video_id': 'CPCWO5177KI', 'text': 'Such an engaging conversation Monroe Mann has. Fantastic', 'author': '@JeffKessler-bf8fj', 'published_date': '2024-05-31T17:25:29Z'}, 'UgygkQzGNmpsemxIm114AaABAg': {'id': 'UgygkQzGNmpsemxIm114AaABAg', 'video_id': '7ooC5Qdavvg', 'text': 'You are truly an inspiration for all of us. Thank you so much 🙏😊', 'author': '@0591242488', 'published_date': '2024-05-31T17:23:41Z'}, 'UgzkyeQ2QgI1nlhPpll4AaABAg': {'id': 'UgzkyeQ2QgI1nlhPpll4AaABAg', 'video_id': 'cBx7b4cT9yw', 'text': 'Parabéns muito obrigada por seu acolhimento.❤.', 'author': '@teresafernandes166', 'published_date': '2024-05-31T17:23:06Z'}, 'UgyxqUHWy3Z4Km9puA54AaABAg': {'id': 'UgyxqUHWy3Z4Km9puA54AaABAg', 'video_id': '4TMPXK9tw5U', 'text': 'Thank you for this', 'author': '@mohameddiallo6108', 'published_date': '2024-05-31T17:21:29Z'}, 'UgwINRssI0a4vzdzoPB4AaABAg': {'id': 'UgwINRssI0a4vzdzoPB4AaABAg', 'video_id': 'da1vvigy5tQ', 'text': 'I went full carnivore, called the lion diet.  I only eat meat, salt, and water.  I have never, and this is not even close to exaggeration, felt better in my life.  Maybe I felt this good when I was a kid, but I can&#39;t remember.  My type II diabetes is gone, my doctor no longer prescribes my diabetes medication even though he said you cannot reverse it and you will be on medicine forever now, oh, and he absolutely blasted the carnivore diet.  Everything he said would happen to me, accept for the elevated cholesterol, did not happen.  My vitamins are all amazing, even vitamin C and I do not supplement.  I am no longer nervous, I feel very calm and my mind just seems to work so much better.  I have a passion for doing things again, something I had lost.  The depression I have been in and out of, but mostly in, for the past 20 years is completely gone.  Not in and out, my depression is just gone.  The only down side is, I do miss carbs, they taste really good.', 'author': '@vodkarage8227', 'published_date': '2024-05-31T17:20:57Z'}, 'UgyMLGm-VH0-EptC-gR4AaABAg': {'id': 'UgyMLGm-VH0-EptC-gR4AaABAg', 'video_id': 'OCFeCHU3A2Q', 'text': 'A fighter in the true sense. Life is a test for everyone in which few pass with flying colours and fewer enjoy it. Salute to Lt Col Man Singh Jamwal Sahab.', 'author': '@VikramadityaSingh', 'published_date': '2024-05-31T17:20:02Z'}, 'UgwxXApGspWBDLAMGwV4AaABAg': {'id': 'UgwxXApGspWBDLAMGwV4AaABAg', 'video_id': 'CPCWO5177KI', 'text': 'The energy and the passion is palpable! It keeps us hooked and want to know everything about the idea. It&#39;s a pleasure to watch and listen to, an eye opener on climbing the ladder of success, whatever it means to each and everyone of us. Also, love the humor! Beautiful talk, cheers to many more speaker opportunities for Monroe!', 'author': '@AnneBelanger', 'published_date': '2024-05-31T17:18:47Z'}, 'UgwdeydKuTE_CCFp3DJ4AaABAg': {'id': 'UgwdeydKuTE_CCFp3DJ4AaABAg', 'video_id': '_D3SX17aeVw', 'text': 'أعظم من غير السائد من العقائد والمسلمات محمد صلى الله عليه وسلم، كلما ابهرني احد بمثل هذه الأفكار الرائعة للأستاذ فهد أجد أن سيد الخلق سبق الجميع إليها. سيرته العطرة صلى الله عليه وسلم هي المنهج.', 'author': '@saeedghannam8512', 'published_date': '2024-05-31T17:18:28Z'}, 'UgxREuUXjxLqcnzvfDJ4AaABAg': {'id': 'UgxREuUXjxLqcnzvfDJ4AaABAg', 'video_id': 'MnvrZk7dxQk', 'text': 'Tashkente gosudarstvenoy biblyoteke inistituta vostokovedeniya xronyatsya bole 70 tisyach stariniye knigi napisani v arabskom alfavite , na uzb , arab i persedskom yazikax polzuysya', 'author': '@mekoyel350', 'published_date': '2024-05-31T17:17:23Z'}, 'UgyrOi4sXIbiyowsyB94AaABAg': {'id': 'UgyrOi4sXIbiyowsyB94AaABAg', 'video_id': 'CPCWO5177KI', 'text': 'This hits on some great points. I appreciate the research behind this as well. This is something I think about often, especially from my own experiences, which have challenged me. This content came at the perfect time. Great Ted Talk!', 'author': '@acepedro', 'published_date': '2024-05-31T17:16:53Z'}, 'Ugy6dKmUSp7N52HcBCN4AaABAg': {'id': 'Ugy6dKmUSp7N52HcBCN4AaABAg', 'video_id': 'CPCWO5177KI', 'text': 'What an amazing topic, really interesting, I&#39;ll do everything to be a break diver😁', 'author': '@jvini7717', 'published_date': '2024-05-31T17:16:43Z'}, 'UgyYapD21bwf9JFr9Ml4AaABAg': {'id': 'UgyYapD21bwf9JFr9Ml4AaABAg', 'video_id': 'sRwy8z8Fmww', 'text': 'Thank you for sharing your truth! I was recently diagnosed with FND. I am in the USA, and it seems that no one knows what is going on. I was advised to seek physical therapy and get psychological health. I am on a journey now to find out what I am really dealing with and bring awareness to this condition. My diagnosis was made March 2024.', 'author': '@msmommatee1', 'published_date': '2024-05-31T17:16:05Z'}, 'UgwvmROkzSeBnf5SQCV4AaABAg': {'id': 'UgwvmROkzSeBnf5SQCV4AaABAg', 'video_id': '3-FiqgVzXqY', 'text': 'T&#39;es kiffant 😊', 'author': '@orilame88974', 'published_date': '2024-05-31T17:15:24Z'}, 'UgxWFfAPlLPC5C1_iHB4AaABAg': {'id': 'UgxWFfAPlLPC5C1_iHB4AaABAg', 'video_id': 'dcr6UUQJkQg', 'text': 'Illness  me  to deal  all  of  it&#39;s  depression and   mood', 'author': '@shariecebrewster5962', 'published_date': '2024-05-31T17:14:10Z'}}
videos={'cBx7b4cT9yw': {'id': 'cBx7b4cT9yw', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Curados para curar: Como encontrar na adversidade sua maior força? | Tâmara Gomes | TEDxCuiabá', 'description': 'Tâmara Gomes é paciente paliativa e fundou em 2020 o  Life “Curadas para curar ", uma rede de apoio dedicada a mulheres com câncer, focada no suporte emocional. O Life já tocou a vida de mais de 150 mulheres e suas famílias, no estado do Mato Grosso, Distrito Federal e Estados Unidos, provando que, embora não possamos mudar o desfecho de todas as histórias, podemos sim tornar o caminho até lá mais bonito e mais leve. Se emocione também com essa fala tocante e inspiradora.  Tâmara Gomes, 38 anos, advogada, casada e mãe do Lucas.  Encontrou  na adversidade a sua maior força. Diagnosticada com câncer e em tratamento paliativo desde 2019, ela transformou sua jornada em uma fonte de inspiração e apoio para outras. Em 2020, ela deu vida ao Life “Curadas para curar ", uma rede de apoio dedicada a mulheres com câncer, focada no suporte emocional. Até hoje, o Life já tocou a vida de mais de 150 mulheres e suas famílias, no estado do Mato Grosso, Distrito Federal e Estados Unidos, provando que, embora não possamos mudar o desfecho de todas as histórias, podemos sim tornar o caminho até lá mais bonito e mais leve. Esta é Tâmara: uma mulher que, mesmo diante dos desafios, escolheu acolher, amparar e inspirar. This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx', 'published_date': '2024-05-30T16:27:40Z', 'view_count': '115', 'like_count': '15', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '6', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/cBx7b4cT9yw/default.jpg', 'medium': 'https://i.ytimg.com/vi/cBx7b4cT9yw/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/cBx7b4cT9yw/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/cBx7b4cT9yw/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/cBx7b4cT9yw/maxresdefault.jpg'}, 'caption_status': 'false'}, '3icoSeGqQtY': {'id': '3icoSeGqQtY', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'How you can be good at math, and other surprising facts about learning | Jo Boaler | TEDxStanford', 'description': 'You have probably heard people say they are just bad at math, or perhaps you yourself feel like you are not “a math person.” Not so, says Stanford mathematics education professor Jo Boaler, who shares the brain research showing that with the right teaching and messages, we can all be good at math. Not only that, our brains operate differently when we believe in ourselves. Boaler gives hope to the the mathematically fearful or challenged, shows a pathway to success, and brings into question the very basics of how our teachers approach what should be a rewarding experience for all children and adults.\n\nJo Boaler is a professor of mathematics education at Stanford and the co-founder of YouCubed, which provides resources and ideas to inspire and excite students about mathematics. She is also the author of the first massive open online course on mathematics teaching and learning. Her book Experiencing School Mathematics won the Outstanding Book of the Year award for education in Britain. A recipient of a National Science Foundation "early career award"\' she was recently named by BBC as one of the eight educators changing the face of education.\n\nThis talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at http://ted.com/tedx', 'published_date': '2016-05-23T03:13:58Z', 'view_count': '1871191', 'like_count': '39111', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '1318', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/3icoSeGqQtY/default.jpg', 'medium': 'https://i.ytimg.com/vi/3icoSeGqQtY/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/3icoSeGqQtY/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/3icoSeGqQtY/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/3icoSeGqQtY/maxresdefault.jpg'}, 'caption_status': 'true'}, 'GsFzT6IA_hU': {'id': 'GsFzT6IA_hU', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Rocks of History - What legacy will you leave behind? | Lieutenant General Ben Hodges | TEDxWHU', 'description': 'Rocks of History - What legacy will you leave behind?\n\nBefore becoming Commander of the US Army Europe, Lt. Gen. Benjamin Hodges was commanding the NATO Allied Land Forces, was part of the Operation IRAQI FREEDOM and his most recent operational assignment was as Director of Operations in Afghanistan. Holding all these positions thus having a great knowledge about other cultures made him gain a far-sighted opinion about worldwide developments, which distinguishes him to be an interesting speaker for our TEDx  event. \n\nThis talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at http://ted.com/tedx', 'published_date': '2016-04-14T17:54:30Z', 'view_count': '18576', 'like_count': '283', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '45', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/GsFzT6IA_hU/default.jpg', 'medium': 'https://i.ytimg.com/vi/GsFzT6IA_hU/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/GsFzT6IA_hU/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/GsFzT6IA_hU/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/GsFzT6IA_hU/maxresdefault.jpg'}, 'caption_status': 'false'}, '3-FiqgVzXqY': {'id': '3-FiqgVzXqY', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Se libérer du regard des autres en 5 minutes | David Laroche | TEDxGEM', 'description': 'David Laroche s’est fait connaître au début de sa carrière par la diffusion de vidéos d’inspiration visant à aider les personnes à gagner en confiance, accélérer leurs résultats et réaliser leurs objectifs. Aujourd’hui, David est devenu un conférencier, coach et entrepreneur à renommée internationale et mène des recherches sur les thématiques du bien-être, de la performance et de la réussite. Il a personnellement rencontré, interviewé ou coaché plus de 300 personnes qui excellent dans leur domaine et cumule plus de soixante millions de vues de ses vidéos internet. Dans ce talk, David Laroche nous aide à nous libérer du regard d’Autrui pour pouvoir se réaliser et faire ressortir tout notre potentiel. David Laroche is the youngest speaker of France and passionate about success and surpassing oneself.\n\nHis main goal is to help people express the best of themselves and achieve their dreams through great speaking meets and the sharing of his experiences.\n\nFormerly shy, David has developed as great entrepreneur and is now travelling the world to spread the news: "You have a dream, it is possible!!" This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx', 'published_date': '2018-12-07T15:51:18Z', 'view_count': '4595913', 'like_count': '130255', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '3628', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/3-FiqgVzXqY/default.jpg', 'medium': 'https://i.ytimg.com/vi/3-FiqgVzXqY/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/3-FiqgVzXqY/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/3-FiqgVzXqY/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/3-FiqgVzXqY/maxresdefault.jpg'}, 'caption_status': 'true'}, '7ooC5Qdavvg': {'id': '7ooC5Qdavvg', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'From Passion to Purpose  | Jatin Khirbat | TEDxSBSC', 'description': "Jatin Khirbat's TEDx talk explores his journey from aspiring model to successful model mentor, author, and content creator. He reflects on the evolving inclusivity of the fashion industry and the challenges of sustaining a career within it. He emphasizes the importance of following one's passion and maintaining a side income stream to support that passion, using his experience as a certified personal trainer during lean modeling times as an example. He concludes by emphasizing the collective responsibility to create a sustainable future, urging action towards a greener, more ethical world. His talk inspires individuals to pursue their passions while being mindful of the impact of their choices on the environment and society. With a captivating presence and determination, Jatin Khirbat has carved his path in the fashion industry. His journey is marked by perseverance and dedication, overcoming obstacles to achieve success. Having graced numerous runways and campaigns, Jatin Khirbat embodies style and elegance, captivating audiences with his unique charisma and poise. Beyond his modeling career, Jatin is committed to giving back to his community. He actively participates in charitable events and uses his platform to raise awareness about social issues, promoting inclusivity and diversity in the fashion world. This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx", 'published_date': '2024-05-08T15:43:57Z', 'view_count': '245', 'like_count': '40', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '29', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/7ooC5Qdavvg/default.jpg', 'medium': 'https://i.ytimg.com/vi/7ooC5Qdavvg/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/7ooC5Qdavvg/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/7ooC5Qdavvg/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/7ooC5Qdavvg/maxresdefault.jpg'}, 'caption_status': 'false'}, '_D3SX17aeVw': {'id': '_D3SX17aeVw', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'يكفي نسخا مكررة | Fahad Al-Ahmadi | TEDxSulaimanAlrajhiColleges', 'description': 'يتوقف معنا الأستاذ فهد الأحمدي في خمس محطات ليطلعنا على أسرار الإبداع والنجاح.\r\n- 99% من أفكارك أتـت من غـيرك\r\n- الأطفال عـباقرة لا يكفون عن السؤال\r\n- الفـرق بين السؤال والتســـاؤل؟\r\n- كيف يـتـم تـقـييـد عـقـلك؟\r\n- أهـم قاعـدة في الإبـداع\n\nفهد بن عامر الأحمدي.. كاتب يومي متـفرغ ومحـرر لزاوية "حول العالم " اليومية بصحيفة الرياض التي لم تتوقف منذ بداية النشر فيها عام ١٩٩١. كما أنه كاتب متعاون في عدد من الصحف العـربية والقنوات التلفزيونية. وقد نشر 7920 مقالا بنهاية عـام 2013 ــ إضافة إلى كونه مدربا متخصصا في كيفية كتابة المقالة والتقارير الصحفية. ــ لديه سلسلة كتب مثل "الاحتمال الآخر" و"حول العالم" و"حول العالم في88 مقال"\r\nFahad Al- Ahmdi A daily writer and an editor for the daily corner" Around the World" which has been being published since 1991 in Riyadh newspaper. Additionally, he is a participant writer in a number of Arabic newspapers and TV channels. Also, his publications reached up to 7920 at the end of 2013. Moreover, Mr. Al-Ahmdi is a specialist trainer in how to write an essay and press reports. He has many series such as, "Another Probability", "Around the World\' and "Around the World in 88 essays".\n\nThis talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at http://ted.com/tedx', 'published_date': '2016-03-28T21:03:04Z', 'view_count': '96076', 'like_count': '2957', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '85', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/_D3SX17aeVw/default.jpg', 'medium': 'https://i.ytimg.com/vi/_D3SX17aeVw/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/_D3SX17aeVw/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/_D3SX17aeVw/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/_D3SX17aeVw/maxresdefault.jpg'}, 'caption_status': 'false'}, 'V6yixyiJcos': {'id': 'V6yixyiJcos', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': "Math isn't hard, it's a language | Randy Palisoc | TEDxManhattanBeach", 'description': "This talk was given at a local TEDx event, produced independently of the TED Conferences. Is 26% proficiency in math acceptable to you?  That's the question teacher and Synergy Academies Founder Dr. Randy Palisoc asks the TEDxManhattanBeach audience.  With only 26% of U.S. twelfth graders proficient in mathematics, Randy shares his solution: teach math as a language. Putting words back into math lessons enables even the youngest school-age minds to grasp complex concepts, such as fractions, that are traditionally thought of as abstract and difficult to understand.  In his stunningly simple and effective approach, math no longer creates problems for kids but solves them.\n\nRandy Palisoc is a passionate educator, known for making math easy.  A founder of the five-time national award winning Synergy Academies, Randy is proud to identify Synergy as an “in spite of” school.  In spite of the fact that its students were disadvantaged and in spite of the fact that it did not have its own facility and had to pack up its classrooms every single week, his school was the first and only elementary school in South Los Angeles to ever win the National Blue Ribbon Award, and was named the #1 Urban Elementary School in America in 2013.\r\n\r\nSeeking a broader impact, Randy left Synergy this year to start Ironbox Education.  One of the ways he believes we can dramatically improve America’s schools is by doing a better job teaching math to our students in a way that makes sense to them.  Randy believes implementing a few changes in the way we approach math will give students the confidence and thinking skills to be more competitive in a global economy.\r\n\r\nFavorite TED Talk:  “My Invention That Made Peace With Lions”: Richard Turere\r\n\r\nThe “Game Changer” he most admires is Steve Jobs, because he had the ability to envision things we did not even know we needed or wanted.\r\n\n\nAbout TEDx, x = independently organized event In the spirit of ideas worth spreading, TEDx is a program of local, self-organized events that bring people together to share a TED-like experience. At a TEDx event, TEDTalks video and live speakers combine to spark deep discussion and connection in a small group. These local, self-organized events are branded TEDx, where x = independently organized TED event. The TED Conference provides general guidance for the TEDx program, but individual TEDx events are self-organized.* (*Subject to certain rules and regulations)", 'published_date': '2014-12-06T01:15:31Z', 'view_count': '3452930', 'like_count': '98478', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '4019', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/V6yixyiJcos/default.jpg', 'medium': 'https://i.ytimg.com/vi/V6yixyiJcos/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/V6yixyiJcos/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/V6yixyiJcos/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/V6yixyiJcos/maxresdefault.jpg'}, 'caption_status': 'true'}, '-l_oc8Ocrig': {'id': '-l_oc8Ocrig', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Longevity is near – and what you can do with it | Tobias Reichmuth | TEDxBielBienne', 'description': 'In this TEDxBielBienne talk, Tobias Reichmuth explores the concept of longevity, discussing how our increasing lifespan presents both challenges and opportunities for society. He offers insights into the latest scientific and technological advancements in anti-aging research and how they could transform the way we live our lives.  Overall, his talk encourages us to embrace the potential of a longer lifespan and consider the implications it has for our personal and societal well-being. Tobias Reichmuth is a Swiss entrepreneur and investor. He founded his first company while still a student. Reichmuth later spent two years traveling the world raising money for SOS Children’s Villages. He is known for his participation in the TV show “Die Höhle der Löwen” Switzerland. This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx', 'published_date': '2023-03-21T14:45:04Z', 'view_count': '13002', 'like_count': '311', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '19', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/-l_oc8Ocrig/default.jpg', 'medium': 'https://i.ytimg.com/vi/-l_oc8Ocrig/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/-l_oc8Ocrig/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/-l_oc8Ocrig/sddefault.jpg'}, 'caption_status': 'true'}, '4TMPXK9tw5U': {'id': '4TMPXK9tw5U', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'The Battle for Your Time: Exposing the Costs of Social Media | Dino Ambrosi | TEDxLagunaBlancaSchool', 'description': 'Do we truly comprehend how much of our time and attention is given to technology? In his talk, Dino Ambrosi reframes how we think about our relationships to devices, and shares his ideas on how to create healthy digital habits. \n Dino Ambrosi, the founder of Project Reboot, is an expert at guiding teens and young adults to relationships with technology that empower them. While studying at UC Berkeley, he created a popular course called Becoming Tech Intentional which he taught to over 60 of his fellow students who reduced their screen time by an average of over 3 hours per day. After graduating in May, he embarked on a mission to spread the contents of his course to a broader audience. Through school assemblies, workshops, and consulting, he has worked with over 500 students and parents to raise awareness about the addictive potential of our devices, drive conversations around digital wellness, and deliver practical strategies to build healthy digital habits. This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx', 'published_date': '2023-03-06T16:41:23Z', 'view_count': '1771952', 'like_count': '102408', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '1823', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/4TMPXK9tw5U/default.jpg', 'medium': 'https://i.ytimg.com/vi/4TMPXK9tw5U/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/4TMPXK9tw5U/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/4TMPXK9tw5U/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/4TMPXK9tw5U/maxresdefault.jpg'}, 'caption_status': 'true'}, 'da1vvigy5tQ': {'id': 'da1vvigy5tQ', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Reversing Type 2 diabetes starts with ignoring the guidelines | Sarah Hallberg | TEDxPurdueU', 'description': 'Can a person be "cured" of Type 2 Diabetes? Dr. Sarah Hallberg provides compelling evidence that it can, and the solution is simpler than you might think.\n\nDr. Sarah Hallberg is the Medical Director of the Medically Supervised Weight Loss Program at IU Health Arnett, a program she created. She is board certified in both obesity medicine and internal medicine and has a Master’s Degree in Exercise Physiology. She has recently created what is only the second non-surgical weight loss rotation in the country for medical students. Her program has consistently exceeded national benchmarks for weight loss, and has been highly successful in reversing diabetes and other metabolic diseases. Dr. Hallberg is also the co-author of www.fitteru.us, a blog about health and wellness.\r\n\r\nB.S., Kinesiology & Exercise Science, Illinois State University, 1994\r\n\r\nM.S., Kinesiology & Exercise Science, Illinois State University, 1996\r\n\r\nM.D., Des Moines University, 2002\n\nThis talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at http://ted.com/tedx', 'published_date': '2015-05-04T15:17:29Z', 'view_count': '11248996', 'like_count': '196950', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '20488', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/da1vvigy5tQ/default.jpg', 'medium': 'https://i.ytimg.com/vi/da1vvigy5tQ/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/da1vvigy5tQ/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/da1vvigy5tQ/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/da1vvigy5tQ/maxresdefault.jpg'}, 'caption_status': 'true'}, 'sRwy8z8Fmww': {'id': 'sRwy8z8Fmww', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Why is Functional Neurological Disorder so poorly understood? | Steph Blanco | TEDxBrayford Pool', 'description': 'Functional Neurological Disorder is the second most commonly diagnosed condition in neurology, so why is it still poorly understood? Neuropsychology lecturer and doctoral researcher Steph Blanco shares her lived experience and research on the condition. She says; "I am not special or unique for finding these things. I firmly believe that those before me would have found them too if only they had stopped and looked and listened to the real lived experiences of those diagnosed." \n\n Steph Blanco is a skilled and compassionate neuropsychologist dedicated to understanding the intricacies of the brain, body and cognition. With unwavering passion, she has devoted her career to conducting research and giving patients with “hidden conditions” a voice. Stephanie-Roxanne is currently a lecturer at Bishop Grosseteste University, completing her PhD at Nottingham Trent University and is Vice Chair at FND Dimensions. With a strong academic background and extensive research experience, she brings a wealth of knowledge to her practice striving to improve the lives of individuals affected by an array of disorders, empowering them to lead fulfilling and meaningful lives. This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx', 'published_date': '2023-11-13T16:44:19Z', 'view_count': '22003', 'like_count': '833', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '175', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/sRwy8z8Fmww/default.jpg', 'medium': 'https://i.ytimg.com/vi/sRwy8z8Fmww/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/sRwy8z8Fmww/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/sRwy8z8Fmww/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/sRwy8z8Fmww/maxresdefault.jpg'}, 'caption_status': 'true'}, 'CPCWO5177KI': {'id': 'CPCWO5177KI', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Eight words to change your life forever | Monroe Mann | TEDxNYU', 'description': 'Monroe Mann, public speaker and founder of BreakDiving.io, shares a philosophy for success that began 25 years ago and culminated in his PhD research in psychology. The 8-word philosophy he lives by has enabled many people to overcome the odds to accomplish their biggest aspirations, and can be used by anyone, anywhere, in any phase of life, to achieve more and be more. Monroe Mann is the founder of BreakDiving.io, a global, not-for-profit personal growth community, as well as Intertect, an international private investigation and intelligence NGO. Mann has an eclectic background including his nomination for a bronze star for his service in Iraq as an intelligence officer, his multiple adventures as an expat living in Shanghai, and his creation of a real estate investment company in Maine and Florida called Windermere Lodge. Mann is also the author of 10 published books, with a PhD in psychology focused on the decision-making processes of successful entertainers. Mann currently lives in Miami Beach, Florida, and is coding a new contact management application for busy well-connected people called, “Contact Points”. This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx', 'published_date': '2024-05-31T15:30:37Z', 'view_count': '68', 'like_count': '28', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '12', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/CPCWO5177KI/default.jpg', 'medium': 'https://i.ytimg.com/vi/CPCWO5177KI/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/CPCWO5177KI/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/CPCWO5177KI/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/CPCWO5177KI/maxresdefault.jpg'}, 'caption_status': 'false'}, 'MnvrZk7dxQk': {'id': 'MnvrZk7dxQk', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Исторические мифы. Откуда они берутся и что с ними делать | Жаксылык Сабитов | TEDxAstana', 'description': 'Жаксылык Сабитов — историк-медиевист. Как историку ему часто приходится сталкиваться с историческими мифами: «Чингиз-хан — казах», «Настоящие казахи – голубоглазые блондины» и так далее. Профессиональному историку разоблачить эти мифы нетрудно, но обыватели с охотой в них верят.\nЖаксылык считает, что мифы отражают запросы общества и касаются прежде всего тех эпох, которые слабо изучены. Проблемы исторической науки в Казахстане не дают заполнить эти лакуны, из-за чего появляются новые мифы. Среди этих проблем отсутствие научных дискуссий, слабая работа с первоисточниками, отсутствие специалистов и непрестижность профессии историка. \n«С развитием исторической науки мифы станут более приближенными к исторической реальности и, возможно, менее взрывоопасными и токсичными», — уверен ученый. \n\nЖақсылық Сәбитов – ортағасыр тарихшысы. Ол өз қызметі аясында «Шыңғыс хан – қазақ», «Нағыз қазақ – көк көзді аққұбалар» деген сарындағы тарихи мифтерге жиі ұшырасады. Кәсіби тарихшыға бұл аңыздардың ақ-қарасын ажырату қиын емес, десе де адамдар оған сенгісі кеп тұра береді.\nЖақсылық мифтер қоғамдағы қажеттіліктерді көрсететінін және көбіне аз зерттелген дәуірлерден бастау алатынын алға тартады. Қазақстандағы тарих ғылымының мәселелері бұл олқылықтардың орнын толтыруға мүмкіндік бермейді, сондықтан жаңа мифтер пайда бола бермек. Мәселелер көзі ғылыми талқылаулардың аздығы мен, негізгі дереккөздермен жұмыстың нашарлығында, мамандардың аздығы мен қоғамда тарихшының беделі болмауында жатыр.\n«Тарих ғылым ретінде дамыған сайын мифтер тарихи шындық шеңберінен шықпайтын болады. Әсері мен ықпалы да төмендей түседі» деген сенімде ғалым. Жаксылык Сабитов\nИсторик-медиевист\nЖаксылык – известный историк, чей индекс Хирша – 8 и который входит в топ-3 казахстанских историка по наукометрическим базам данных. По роду своей деятельности Жаксылык постоянно сталкивается с различными историческими мифами, которые широко распространены в нашем обществе. Ему разоблачить их нетрудно, но обыватели с охотой в них верят. Жаксылык считает, что мифы отражают запросы общества и касаются прежде всего тех эпох, которые слабо изучены. И решением этого может стать развитие исторической науки.\n\nЖақсылық Сәбитов\nОртағасыр тарихшысы, танымал маман. Ғылыми-метрикалық мәліметтер базасы бойынша қазақстандық тарихшылардың үздік үштігіне енеді, Хирш индексі – 8. Жақсылық біздің қоғамда кең тараған түрлі тарихи мифтерге жиі ұшырасады. Оларды жоққа шығару тарихшы үшін еш қиын емес, алайда адамдар мұндай ақпаратқа бәрібір сенгісі келіп тұрады. Жақсылық мифтер қоғамның қажеттіліктерін көрсететінін және көбіне аз зерттелген дәуірлерге қатысты екенін алға тартады. Оны зерттеу арқылы тарих ғылымын ілгерілетуге болады. This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx', 'published_date': '2023-01-20T15:47:40Z', 'view_count': '39921', 'like_count': '1899', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '570', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/MnvrZk7dxQk/default.jpg', 'medium': 'https://i.ytimg.com/vi/MnvrZk7dxQk/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/MnvrZk7dxQk/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/MnvrZk7dxQk/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/MnvrZk7dxQk/maxresdefault.jpg'}, 'caption_status': 'false'}, 'dcr6UUQJkQg': {'id': 'dcr6UUQJkQg', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'Broken and Brave: Overcoming Mental Illness | Claudia Cornelison | TEDxGeorgiaStateU', 'description': 'What if bravery was not reserved for the heroes who come rushing in to rescue the day? What if we appreciated the bravery of those who are fighting mental illness every day? Educators, parents and friends are encouraged to come alongside those battling mental illness in small acts of bravery. Claudia Cornelison is a political science student, published poet, a proud member of Georgia State University’s mock trial team. She has struggled with several forms of mental illness— including depression and anxiety— and hopes to use what she has learned from those struggles to help other college students and young adults. She owes much of her inspiration to her supportive family, her loving boyfriend, and her forever loyal Goldendoodle, Boaz. This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx', 'published_date': '2018-12-13T16:21:08Z', 'view_count': '11835', 'like_count': '268', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '27', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/dcr6UUQJkQg/default.jpg', 'medium': 'https://i.ytimg.com/vi/dcr6UUQJkQg/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/dcr6UUQJkQg/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/dcr6UUQJkQg/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/dcr6UUQJkQg/maxresdefault.jpg'}, 'caption_status': 'false'}, 'OCFeCHU3A2Q': {'id': 'OCFeCHU3A2Q', 'playlist_id': 'UUsT0YIqwnpJCM-mx7-gSA4Q', 'channel_id': 'UCsT0YIqwnpJCM-mx7-gSA4Q', 'name': 'If you are still living your task is not over | Lt.Col.Man Singh | TEDxMIETJammu', 'description': 'NOTE FROM TED: This talk contains a graphic description of war injuries which some viewers may find disturbing. TEDx events are independently organized by volunteers. The guidelines we give TEDx organizers are described in more detail here: http://storage.ted.com/tedx/manuals/tedx_content_guidelines.pdf\n\nLt. Col. Man Singh Jamwal shared his remarkable journey of resilience and determination, and his\nstory was a beacon of hope for all who faced their battles, a reminder that within each of us\nlies the power to overcome, no matter the odds. He shared his two near death experiences and how he defied all odds and managed to survive. I, Lt. Col. Man Singh Jamwal, passed Matric from St. Mary\'s Presentation Convent in 1970,\npassed P.U.C in 1971, joined National defence academy in 1971 and passed out in 1974\nand commissioned from Indian Military Academy, Dehradun on 15th June , 1975. Served\n23years in Army and 12 years as Zila Sanik Welfare Officer. Topic of talk- "if you are still\nliving your task is not over" correlating it to life experiences This talk was given at a TEDx event using the TED conference format but independently organized by a local community. Learn more at https://www.ted.com/tedx', 'published_date': '2024-05-23T14:40:24Z', 'view_count': '84', 'like_count': '10', 'dislike_count': 0, 'favorite_count': '0', 'comment_count': '4', 'duration': 0, 'thumbnails': {'default': 'https://i.ytimg.com/vi/OCFeCHU3A2Q/default.jpg', 'medium': 'https://i.ytimg.com/vi/OCFeCHU3A2Q/mqdefault.jpg', 'high': 'https://i.ytimg.com/vi/OCFeCHU3A2Q/hqdefault.jpg', 'standard': 'https://i.ytimg.com/vi/OCFeCHU3A2Q/sddefault.jpg', 'maxres': 'https://i.ytimg.com/vi/OCFeCHU3A2Q/maxresdefault.jpg'}, 'caption_status': 'false'}}

# End Mock
from tools import load_db_cfg
dao = YtDao(load_db_cfg('cfg.json'))
dao.add_channel_info(channel,playlists,comments,videos)
dao.close()

Successfully connected to 'youtube' database!! 
